**Q2) UNIGRAM**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import os
import pickle
import string
import re
import nltk

from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords

nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('omw-1.4')

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...


True

In [3]:
import os
import pickle

data_folder = "/content/drive/MyDrive/IR/text_files"
files_list = os.listdir(data_folder)
data = {}

for current_file in files_list:
    with open(os.path.join(data_folder, current_file), "r") as file_content:
        data[current_file] = file_content.read()


inverted_index_dict = {}

for current_file, file_text in data.items():
    words = file_text.split()

    for word in words:
        if word not in inverted_index_dict:
            inverted_index_dict[word] = set()

        inverted_index_dict[word].add(current_file)

inverted_index_dict = {word: list(files) for word, files in inverted_index_dict.items()}



with open("/content/drive/MyDrive/Q2_pickle.pkl", "wb") as file_obj:
    pickle.dump(inverted_index_dict, file_obj)


with open("/content/drive/MyDrive/Q2_pickle.pkl", "rb") as file_obj:
    loaded_unigram_inverted_index = pickle.load(file_obj)

print(inverted_index_dict)

{'price': ['file458.txt', 'file989.txt', 'file159.txt', 'file370.txt', 'file20.txt', 'file637.txt', 'file543.txt', 'file860.txt', 'file231.txt', 'file755.txt', 'file265.txt', 'file470.txt', 'file488.txt', 'file911.txt', 'file224.txt', 'file162.txt', 'file4.txt', 'file30.txt', 'file33.txt', 'file64.txt', 'file126.txt', 'file991.txt', 'file681.txt', 'file888.txt', 'file850.txt', 'file432.txt', 'file906.txt', 'file246.txt', 'file513.txt', 'file220.txt', 'file270.txt', 'file234.txt', 'file965.txt', 'file28.txt', 'file102.txt', 'file775.txt', 'file586.txt', 'file489.txt', 'file550.txt', 'file870.txt', 'file183.txt', 'file486.txt', 'file329.txt', 'file768.txt', 'file205.txt', 'file152.txt', 'file314.txt', 'file854.txt', 'file347.txt', 'file124.txt', 'file736.txt', 'file925.txt', 'file885.txt', 'file629.txt', 'file544.txt', 'file907.txt', 'file689.txt', 'file701.txt', 'file742.txt', 'file381.txt', 'file872.txt', 'file242.txt', 'file453.txt', 'file831.txt', 'file519.txt', 'file765.txt', 'file6

Load Pickle file and save pickle file

---



---



---



In [4]:
def load_pickle(filename):
    with open(filename, 'rb') as f:
        return pickle.load(f)

def save_pickle(obj, filename):
    with open(filename, 'wb') as f:
        pickle.dump(obj, f)

In [5]:
print(len(inverted_index_dict))

6471


calculation for Operation

---



In [14]:
def union(file1, file2):
    result = file1 + [item for item in file2 if item not in file1]
    return result

def intersect(file1, file2):
    return [item for item in file1 if item in file2]

def complement(file1, all_docs):
    return [item for item in all_docs if item not in file1]

def andd(file1, file2):
    x = y = 0
    file1.sort()
    file2.sort()

    for item1 in file1:
        value1 = int(item1[4:-4])

        while y < len(file2):
            value2 = int(file2[y][4:-4])

            if value1 == value2:
                # num_comparisons += 1
                y += 1
                break
            elif value1 < value2:
                break
            else:
                y += 1


def orr(file1, file2):
    x = y = 0
    file1.sort()
    file2.sort()

    while x < len(file1) or y < len(file2):
        if x < len(file1) and y < len(file2):
            value1 = int(file1[x][4:-4])
            value2 = int(file2[y][4:-4])

            if value1 == value2:
                x += 1
                y += 1
            elif value1 < value2:
                x += 1
            else:
                y += 1
        elif x < len(file1):
            x += 1
        else:
            y += 1


def docs(query, operations, unigram_index, all_files):
    lem = WordNetLemmatizer()
    query = query.lower()
    query = re.sub('[^A-Z a-z ]+', ' ', query)
    query = query.split()
    tokens = [j for j in query]
    stop_words = set(stopwords.words("english"))
    tokens = [lem.lemmatize(token) for token in tokens if token not in stop_words and token not in string.punctuation and token.strip()]

    print(tokens)

    combined_elements = []
    for token, operation in zip(tokens, operations):
      combined_elements.extend([token, operation])
    combined_elements.extend(tokens[len(operations):] + operations[len(tokens):])
    query_ans = ' '.join(combined_elements)

    query_tokens = tokens

    res_set = [list(unigram_index[token]) for token in query_tokens if token in unigram_index]

    result = []
    compare = 0
    i = 0

    for operations in operations:
        if res_set and i < len(res_set):
            # print(res_set)
            if operations == "AND":
                if i == 0:
                    compare = andd(res_set[i], res_set[i+1])
                    result = intersect(res_set[i], res_set[i+1])
                    # print("IM' in and")
                    i = i + 1
                else:
                    compare = andd(result, res_set[i])
                    result = intersect(result, res_set[i])
                    i = i + 1

            elif operations == "OR":
                if i == 0:
                    compare = orr(res_set[i], res_set[i+1])
                    result = union(res_set[i], res_set[i+1])
                    i = i + 1
                else:
                    compare = orr(result, res_set[i])
                    result = union(result, res_set[i])
                    i = i + 1

            elif operations == "AND NOT":
                result = list(set(res_set[i])-set(res_set[i+1]))
                i = i + 1

            elif operations == "OR NOT":
                comp_set = list(set(res_set[i+1])-set(res_set[i]))
                result = list(set(all_files)-set(comp_set))
                # print("I'm in or NOt")
                i = i + 1
        else:
            break

    return result, compare, query_ans


def file_names(num_files):
    return [f"file{i + 1}.txt" for i in range(num_files)]

def process_query(query, operations, inverted_index, files):
    result, number_of_comparisons, query_ans = docs(query, operations, inverted_index, files)
    query_statement = " ".join(operations)
    sorted_result = sorted(result)
    return query_statement, len(result), sorted_result, query_ans

if __name__ == "__main__":
    n = int(input("Enter the number of queries: ").strip())
    files = file_names(999)

    for i in range(n):
        query = input(f"Enter query {i + 1}: ").strip()
        operations = input("Enter operations separated by commas: ").strip().split(",")

        query_statement, num_retrieved, retrieved_names, query_ans = process_query(query, operations, loaded_unigram_inverted_index, files)

        # print(f"\nQuery {i + 1}: {query_statement}")
        print(f"\nQuery {i + 1}: {query_ans}")

        print("Number of documents retrieved for query", i + 1, ":", num_retrieved)
        print("Names of the documents retrieved for query", i + 1, ":", retrieved_names)
        print()


Enter the number of queries: 1
Enter query 1: celestion fitted ish cute seltosa
Enter operations separated by commas: AND, AND, AND, OR NOT
['celestion', 'fitted', 'ish', 'cute', 'seltosa']

Query 1: celestion AND fitted  AND ish  AND cute  OR NOT seltosa
Number of documents retrieved for query 1 : 1
Names of the documents retrieved for query 1 : ['file100.txt']

